In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
#from tensorflow.keras import Sequential
#from tensorflow.keras import Dense, Dropout
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, InputLayer, Reshape
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

from keras.utils import np_utils

import matplotlib as mpl
#import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD

In [25]:
# Helper to plot loss
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.legend()
  plt.grid(True)

# Keras and Tensorflow Optimizations

There are several things that we can do to make our networks a bit better. Unfortunately for much of this there aren't definitive answers for "what is the best choice", so we do have to do some trial and error, but we can use some guidelines to get us started in the right direction. 

## Load MNIST Data

We can use the MNIST digit dataset for testing, since it is reasonably large. 

In [22]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
#model = keras.Sequential([
#  keras.layers.InputLayer(input_shape=(28, 28)),
#  keras.layers.Reshape(target_shape=(28, 28, 1)),
#  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
#  keras.layers.MaxPooling2D(pool_size=(2, 2)),
#  keras.layers.Flatten(),
#  keras.layers.Dense(10)
#])
print(train_images.shape)
#train_images = train_images.reshape((train_images.shape[0], 28*28)).astype('float32')
#test_images = test_images.reshape((test_images.shape[0], 28*28)).astype('float32')

train_labels = np_utils.to_categorical(train_labels)
test_labels = np_utils.to_categorical(test_labels)

model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation="softmax"))

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

model.fit(
  train_images,
  train_labels,
  epochs=10,
  validation_split=0.2,
)
print(model.evaluate(test_images, test_labels))

(60000, 28, 28)
Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2202 - accuracy: 0.9356 - val_loss: 0.1135 - val_accuracy: 0.9663
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0891 - accuracy: 0.9729 - val_loss: 0.0992 - val_accuracy: 0.9716
Epoch 3/10
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0554 - accuracy: 0.9829 - val_loss: 0.0909 - val_accuracy: 0.9712
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0384 - accuracy: 0.9883 - val_loss: 0.0767 - val_accuracy: 0.9777
Epoch 5/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.0286 - accuracy: 0.9912 - val_loss: 0.0765 - val_accuracy: 0.9784
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0223 - accuracy: 0.9924 - val_loss: 0.0833 - val_accuracy: 0.9783
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0158 - accuracy: 0.9953 - val_loss: 0.1032

## Prequel - Saving and Loading Models

As we've seen, models can take a long time to train in many cases. Like with the sklearn models, we can save and load ours as they are trained and reused. This is a pretty integral part of making neural network models usable, so it is pretty easy. 

In addition to this we often see models saved in the h5 format, which just saves slightly less stuff along with the model. If we are using models trained elsewhere this format is very common. 

In [ ]:
# Save my model
model.save('model_path')
model = keras.models.load_model('model_path')

In [ ]:
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("my_h5_model.h5")

# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_h5_model.h5")

## Network Size

Probably the first question that we will think of when building networks through Tensorflow is "how big should it be"? This is a very big question, and one of those ones without a real answer. We can put some guidelines in place to help us though. 

### What Does the Size Mean?

The size of a neural network is also known as the capacity. We can relate it roughly to the size of our first model, the tree. The larger a network is the higher its capacity to learn. 

### What Size to Use

We can start with a few guidelines to have a reasonably sized neural network. These steps do not ensure an optimal solution, but they'll get us started. There really is not a prescribed method for calculating the optimal network size (beleive me, I've looked), but there are several rules of thumb we can build together:

<ul>
<li> Start with an input layer that is either
    <ul>
    <li> The width of the data, if the feature set is relatively small. 
    <li> A reasonably large number if the feature set is large. 
    <li> We don't have a true diving line, but 512 is a reasonable value to try for an upper end, at least at first. 
    </ul>
<li> Add 1 or 2 hidden layers of the same size and observe the results. We want to keep the model smaller if making it larger doesn't improve things, so first we shoudl see how good of a job a small model does. If the data is very large, skipping past the 1 layer step may save some time since we can predict that we can do better with a larger model in advance. 
<li> Increase layers of the same size until we get some overfitting and the training loss flattens. We want to reach the point where the model is getting to be excellent at predicting the training data. This is something we can see in the plot by noticing that the validation loss flatlines or starts to get worse. The training loss flattening is an indication that the model is not getting any better at learning the training data; we can use early stopping with a loose patience setting on training loss and lots of epochs to find this. 
<li> Add regularization steps to cut down that overfitting. We can try regularization and dropouts to cut down on that overfitting. We probably want to try a few options, parameters, and combinations here, there's not really a way to know in advance which regularization will work best on our data. 
<li> "Funnel" the layer size, potentially adding more layers. The traditional configuration of layers is to gradually decrease the size from the input layer towards the output layer. There is open debate on if this is better than having layers that are all the same size. We can play with this a little to see if results improve or not. 
<li> Use pruning. Much like a tree we can prune back a model to fight overfitting. 
</ul>

### Height vs Width

Another begged question is should we make networks wider (more neurons) or deeper (more layers)? Once again, there's no universal answer, but the general evidence leans towards more layers. There are several reasons for this, none of them definitive, but taken as a whole they add up to a strong case:

<ul>
<li> Ability to learn different representation of the data - this will be more clear next time when we start to look at some image specific neural networks, but one of the cool features of neural networks is that at each layer the network "sees" a different representation of the data, as it goes through each round of transformations. This has the effect of allowing it to identify different features at each layer, and use those features to make more and more accurate predictions. We'll examine this more soon. 
<li> Avoiding overfitting - extremely wide neural networks tend towards overfitting the training data and not generalizing as well to new data. 
<li> Ability to add interim steps - with a multi layer network we can add multiple steps such as regularization or dropouts, again to fight overfitting. 
<li> Automatic feature selection - deep neural networks will automatically perform a type of feature selection as the least important features are minimized in their importance. This is an emerging area of research - some people have argued that well designed neural networks can remove the need for feature selection, and neural networks are being created to be feature selection tools. We can see this illustrated most clearly with images again, we feed a network an entire image, and get a prediction. Note that this isn't a total rejection of feature selection for neural networks, improving the feature set will impact neural network models just as it will for ordinary models; with neural networks we just have the potential for the network to "cover for mistakes" in the features. This is more dramatic as data size and network size increase. 
<li> Results - deep learning has become a common term recently for a reason, due to the success of deep neural networks with many layers. Most of the cool stuff that we see coming from AI such as image recognition, translation, and self navigating robots are the result of deep learning networks. In practice these networks have tended to outperform shallower ones, especially in more complex tasks. 
</ul>

Why not make a model that is both very wide and very deep? This will tend to overfit as it can "memorize" the training data. With large datasets we do see very large models in some cases, since the more data we have, the more fitting we can handle. 

In [ ]:
# Big Model
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(784, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(784, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

model.fit(
  train_images,
  train_labels,
  epochs=10,
  validation_split=0.2,
)

In [23]:
# Big Model
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

model.fit(
  train_images,
  train_labels,
  epochs=10,
  validation_split=0.2,
)

Epoch 1/40
1500/1500 [==============================] - 10s 6ms/step - loss: 0.2887 - accuracy: 0.9139 - val_loss: 0.1424 - val_accuracy: 0.9595
Epoch 2/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1444 - accuracy: 0.9584 - val_loss: 0.1212 - val_accuracy: 0.9641
Epoch 3/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1184 - accuracy: 0.9659 - val_loss: 0.1212 - val_accuracy: 0.9678
Epoch 4/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1026 - accuracy: 0.9715 - val_loss: 0.1203 - val_accuracy: 0.9699
Epoch 5/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0886 - accuracy: 0.9753 - val_loss: 0.0921 - val_accuracy: 0.9758
Epoch 6/40
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0807 - accuracy: 0.9779 - val_loss: 0.1048 - val_accuracy: 0.9741
Epoch 7/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0738 - accuracy: 0.9802 - val_loss: 0.1194 - val_accurac

In [24]:
# Tapered Model
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(350, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

model.fit(
  train_images,
  train_labels,
  epochs=10,
  validation_split=0.2,
)

Epoch 1/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3825 - accuracy: 0.8887 - val_loss: 0.1608 - val_accuracy: 0.9557
Epoch 2/40
1500/1500 [==============================] - 8s 6ms/step - loss: 0.1757 - accuracy: 0.9555 - val_loss: 0.1435 - val_accuracy: 0.9614
Epoch 3/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1398 - accuracy: 0.9637 - val_loss: 0.1063 - val_accuracy: 0.9737
Epoch 4/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1158 - accuracy: 0.9696 - val_loss: 0.1220 - val_accuracy: 0.9698
Epoch 5/40
1500/1500 [==============================] - 8s 6ms/step - loss: 0.1016 - accuracy: 0.9734 - val_loss: 0.1057 - val_accuracy: 0.9733
Epoch 6/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0905 - accuracy: 0.9768 - val_loss: 0.0978 - val_accuracy: 0.9774
Epoch 7/40
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0805 - accuracy: 0.9794 - val_loss: 0.0986 - val_accuracy:

## Epochs and Batch Sizes

### Epochs

Each epoch is a run through all of the training data. Epochs are simple, we can set a large number and use early stopping to cut things off when we've reached the best result. 

### Batch Sizes

Batch size determines how many records are processed before the gradients are updated - i.e. the number of records between one forward and backwards pass.

The batch sizes are a matter of very open debate for the optimal solution. At the high end, batch sizes are limited by what can fit in memory. When dealing with very large data this may matter as a batch that is a small fraction of the data may be a massive absolute size. At the lower end using smaller batches gives the same effect as it does when we looked at regular gradient descent - the gradients become less stable as we are relying on a smaller number of records. In reading more about batch sizes I want to update my recommendation to be even smaller than the 50 to 150 I suggested before, down to less than 100, even as small as into the single digits. There is research that smaller batch sizes tend to produce models that generalize better than ones with larger batches. 

Larger batch sizes do tend to be processed more quickly, sometimes substantially so, as the hardware is better able to be "saturated" with data to process. 

Dont' stress too much on batch size, this is really something that needs to be grid searched to find a great answer. 

In [26]:
# Big Batch
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(350, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

model.fit(
  train_images,
  train_labels,
  epochs=10,
  batch_size=5000,
  validation_split=0.2,
)

Epoch 1/10
10/10 [==============================] - 1s 53ms/step - loss: 1.3543 - accuracy: 0.6221 - val_loss: 0.4702 - val_accuracy: 0.8680
Epoch 2/10
10/10 [==============================] - 0s 28ms/step - loss: 0.4829 - accuracy: 0.8537 - val_loss: 0.3105 - val_accuracy: 0.9085
Epoch 3/10
10/10 [==============================] - 0s 28ms/step - loss: 0.3535 - accuracy: 0.8961 - val_loss: 0.2437 - val_accuracy: 0.9295
Epoch 4/10
10/10 [==============================] - 0s 30ms/step - loss: 0.2751 - accuracy: 0.9197 - val_loss: 0.2063 - val_accuracy: 0.9403
Epoch 5/10
10/10 [==============================] - 0s 30ms/step - loss: 0.2300 - accuracy: 0.9331 - val_loss: 0.1785 - val_accuracy: 0.9482
Epoch 6/10
10/10 [==============================] - 0s 31ms/step - loss: 0.1980 - accuracy: 0.9423 - val_loss: 0.1580 - val_accuracy: 0.9525
Epoch 7/10
10/10 [==============================] - 0s 30ms/step - loss: 0.1723 - accuracy: 0.9497 - val_loss: 0.1449 - val_accuracy: 0.9568
Epoch 8/10
10

In [27]:
# Small Batch
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(350, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

model.fit(
  train_images,
  train_labels,
  epochs=10,
  batch_size=2,
  validation_split=0.2,
)

Epoch 1/10
24000/24000 [==============================] - 106s 4ms/step - loss: 0.3083 - accuracy: 0.9146 - val_loss: 0.1766 - val_accuracy: 0.9500
Epoch 2/10
24000/24000 [==============================] - 105s 4ms/step - loss: 0.2268 - accuracy: 0.9441 - val_loss: 0.1681 - val_accuracy: 0.9578
Epoch 3/10
24000/24000 [==============================] - 117s 5ms/step - loss: 0.2132 - accuracy: 0.9523 - val_loss: 0.2054 - val_accuracy: 0.9598
Epoch 4/10
24000/24000 [==============================] - 106s 4ms/step - loss: 0.2119 - accuracy: 0.9539 - val_loss: 0.1963 - val_accuracy: 0.9642
Epoch 5/10
24000/24000 [==============================] - 130s 5ms/step - loss: 0.1973 - accuracy: 0.9569 - val_loss: 0.1966 - val_accuracy: 0.9668
Epoch 6/10
24000/24000 [==============================] - 107s 4ms/step - loss: 0.2046 - accuracy: 0.9590 - val_loss: 0.1686 - val_accuracy: 0.9663
Epoch 7/10
24000/24000 [==============================] - 106s 4ms/step - loss: 0.1919 - accuracy: 0.9614 - val_

## Optimizer

Of all options the optimizer is the one we will care about the least. Each different optimizer is a different algorithm for doing the gradient descent. The optimizers have different results with respect to speed, memory usage, computational expense, and likelyhood to get stuck in a local minima. 

Adam is a good compromise between all factors and is very commonly used. We'll just use this for our work. One other common one is RMSprop, if you're feeling spicy, give that a try and see if there are any imporvements. 

## Activation 

Activation functions are the key to adding non-linearity to the network allowing it to learn complex and non-linear relationships in the data. We've used ReLU as the default and that is a solid choice in most cases. 

ReLU has one issue, the dying ReLU problem. This can happen when we get inputs to the activation function fall in the negative area. In short there can be neurons that "die" and never get updated again. 

To combat the dying ReLU problem there are a couple of other activation functions that avoid that issue - Leaky ReLU and ELU. Each one changes the negative values to something other than 0 - Leaky ReLU uses a slight linear gradient, ELU uses an exponential function for a similar, but curved, slight gradient. 

In [31]:
# Take a leak 
model = keras.Sequential()
model.add(InputLayer(input_shape=(28, 28)))
model.add(Flatten())
model.add(Dense(500, activation='leaky_relu'))
model.add(Dropout(0.2))
model.add(Dense(350, activation='leaky_relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

# Train the digit classification model
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics='accuracy')

model.fit(
  train_images,
  train_labels,
  epochs=10,
  batch_size=64,
  validation_split=0.2,
)

Epoch 1/10
24000/24000 [==============================] - 114s 5ms/step - loss: 0.4397 - accuracy: 0.8867 - val_loss: 0.3563 - val_accuracy: 0.9218
Epoch 2/10
24000/24000 [==============================] - 104s 4ms/step - loss: 0.4086 - accuracy: 0.9161 - val_loss: 0.5587 - val_accuracy: 0.9029
Epoch 3/10
24000/24000 [==============================] - 105s 4ms/step - loss: 0.4283 - accuracy: 0.9231 - val_loss: 0.3020 - val_accuracy: 0.9463
Epoch 4/10
24000/24000 [==============================] - 102s 4ms/step - loss: 0.4715 - accuracy: 0.9267 - val_loss: 0.2907 - val_accuracy: 0.9537
Epoch 5/10
24000/24000 [==============================] - 115s 5ms/step - loss: 0.4909 - accuracy: 0.9324 - val_loss: 0.4184 - val_accuracy: 0.9472
Epoch 6/10
24000/24000 [==============================] - 103s 4ms/step - loss: 0.5237 - accuracy: 0.9354 - val_loss: 0.4313 - val_accuracy: 0.9508
Epoch 7/10
24000/24000 [==============================] - 112s 5ms/step - loss: 0.5526 - accuracy: 0.9358 - val_

## Initialization

The initialization provides the starting point for all the weights and bias values that we start out with. We initially started with random values in the scratch network - this is generally fine, but we can sometimes do better. 

### Imbalanced Weighting

One application where initialization can help significantly is when dealing with imbalanced data. 

In [29]:
file = tf.keras.utils
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
raw_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [30]:
neg, pos = np.bincount(raw_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 284807
    Positive: 492 (0.17% of total)



### We Have an Imbalance

In [32]:
cleaned_df = raw_df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps = 0.001 # 0 => 0.1¢
cleaned_df['Log Ammount'] = np.log(cleaned_df.pop('Amount')+eps)

In [33]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [34]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (182276,)
Validation labels shape: (45569,)
Test labels shape: (56962,)
Training features shape: (182276, 29)
Validation features shape: (45569, 29)
Test features shape: (56962, 29)


In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

## Pruning

We can also use pruning to improve our networks, which is built into Tensorflow and is similar in concept to the tree pruning we did earlier. 

### Smaller Models

One other consideration is that we can use pruning to create smaller models that are better able to be executed on weaker hardware. In the context of a full computer, creating a prediction with a neural network is pretty fast. If we want to move the model to small embedded devices though, the memory and processing needs can still be excessive. 

Some scenarios where this comes up are things like security cameras that can recognize images, robots that can navigate themselves, or evern small computers like a Raspberry Pi. This challenge is magnified if you are dealing with something like video, which can generate 30+ images per second. Small models that are almost as good, but can be run with less compouting power allow the power of neural networks to be expanded to more devices - train on a powerful computer, us on a small and weak computer. 

In [36]:
!pip install tensorflow_model_optimization

     |████████████████████████████████| 237 kB 5.0 MB/s 


In [37]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

model_for_pruning = prune_low_magnitude(model)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_flatten  (None, 784)              1         
 _16 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_dense_5  (None, 500)              784502    
 6 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dropout  (None, 500)              1         
 _13 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_dense_5  (None, 350)              350352    
 7 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dropout  (None, 350)            

In [ ]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=64, epochs=10, validation_split=.2,
                  callbacks=callbacks)

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)